In [1]:
from lighttag_to_adm import convert_lighttag_example_to_adm
from adm_to_lighttag import adm_doc_to_lighttag_suggetions,adm_document_list_to_lighttag_suggestions
import json
import os
import pandas as pd

## Get The ADMS
In this case we'll just make adms from LightTag data. You probably have ADMs in a file or in memory

## Notes on the adm_document_list_to_lighttag_suggestions function
This function takes two optional callback arguments

* **example_id_fn** extracts the lighttag example id from the ADM Doc

* **lighttag_tag_name_extractor** resolves the LightTag tag name of the entity

Both of these have defaults defined in adm_to_lighttag


In [2]:
adms = []
for root,d,files in os.walk('/tmp/sixgill/'):
    for f in files:
        if f.endswith('adm.json'):
            adms.append(json.load(open(os.path.join(root,f))))
            
seen_examples = adm_document_list_to_lighttag_suggestions(adms)

In [3]:
suggestions = seen_examples['suggestions']

seen_example_ids = seen_examples['seen_example_ids']

# TODO For Sixgill/ Nadav
 * Notice that the example_id is a list, please modify the example_id_fn to match your branch
 * Implement the lighttag_tag_name_extractor to get the lighttag tag name
 * Is Basis sending more tags than what is in your schema ? We'll reject those 

In [ ]:
pd.DataFrame(suggestions)

# Sanity Check conflicts which LightTag might reject

We'll run the conflict resolveer, if you get any warnings here than you need to check why you have overlapping suggestions. 

In [ ]:
import copy
from itertools import groupby
pseudo_annotations = []
for sug in suggestions:
    x = copy.copy(sug)
    x['annotated_by'] = ["my_model"]
    pseudo_annotations.append(x)

pseudos_by_example = groupby(pseudo_annotations,key=lambda x:x['example_id'])


In [ ]:
from conflict_resolver import resolve_annotation_conflicts
for example_id, example_suggestions in pseudos_by_example:
        resolve_annotation_conflicts(list(example_suggestions))


## Now Upload The Suggestions as Usual

From this point, you need to [register the model in LightTag](https://guide.lighttag.io/suggestions/suggestions/#2.-Registering-a-SuggestionModel) and [upload the suggestions ](https://guide.lighttag.io/suggestions/suggestions/#4.-Upload-Your-Suggestions)